In [1]:
import os
import sys

sys.path.insert(0, f'{os.environ.get("HOME")}/workspace/recommendation-study')

In [2]:
from util.data import DataLoader

dataset = DataLoader().load()

In [3]:
movie_content = dataset.item_content

movie_content['tag_genre'] = movie_content['tag'].fillna('').apply(list) + movie_content['genre'].fillna('').apply(list)
movie_content['tag_genre'] = movie_content['tag_genre'].apply(lambda x: list(map(str, x)))

tag_genre_data = movie_content.tag_genre.tolist()
tag_genre_data[10]

['girlie movie',
 'politics',
 'dvd collection',
 'politics',
 'president',
 '23.03.06',
 'bibliothek',
 'seen',
 'cute',
 'seen more than once',
 'drama',
 'romance',
 'amazing kung fu',
 'decent flick',
 'president',
 'us president',
 'white house',
 'president',
 'aaron sorkin',
 'Comedy',
 'Drama',
 'Romance']

In [4]:
import gensim
import logging
from gensim.corpora.dictionary import Dictionary

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

common_dictionary = Dictionary(tag_genre_data)
common_corpus = [common_dictionary.doc2bow(text) for text in tag_genre_data]

print(common_corpus)

2023-08-07 22:11:34,177 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2023-08-07 22:11:34,248 : INFO : adding document #10000 to Dictionary<14749 unique tokens: ['3d', 'Adventure', 'Animation', 'Children', 'Comedy']...>
2023-08-07 22:11:34,252 : INFO : built Dictionary<15261 unique tokens: ['3d', 'Adventure', 'Animation', 'Children', 'Comedy']...> from 10681 documents (total 117144 corpus positions)
2023-08-07 22:11:34,252 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<15261 unique tokens: ['3d', 'Adventure', 'Animation', 'Children', 'Comedy']...> from 10681 documents (total 117144 corpus positions)", 'datetime': '2023-08-07T22:11:34.252819', 'gensim': '4.3.1', 'python': '3.9.9 (main, May 31 2023, 19:14:41) \n[Clang 14.0.3 (clang-1403.0.22.14.1)]', 'platform': 'macOS-13.2.1-arm64-arm-64bit', 'event': 'created'}


[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 2), (11, 18), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 3), (20, 2), (21, 2), (22, 1), (23, 3), (24, 4), (25, 1), (26, 10), (27, 1), (28, 1), (29, 1), (30, 2), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 29), (45, 1), (46, 1), (47, 1), (48, 1), (49, 3), (50, 3), (51, 2), (52, 2), (53, 5), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1)], [(1, 1), (3, 1), (5, 1), (20, 1), (32, 4), (50, 2), (68, 1), (69, 3), (70, 3), (71, 1), (72, 1), (73, 1), (74, 2), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 8), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1)], [(4, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 2), (95, 2), (96, 1), (97, 1), (98, 1)], [(4, 1), (88, 1), (99, 1), (100, 1)], [(4, 1), (30

In [5]:
common_dictionary[10]

'animated'

In [6]:
lda_model = gensim.models.LdaModel(common_corpus, id2word=common_dictionary, num_topics=50, passes=30)

2023-08-07 22:11:34,333 : INFO : using symmetric alpha at 0.02
2023-08-07 22:11:34,333 : INFO : using symmetric eta at 0.02
2023-08-07 22:11:34,335 : INFO : using serial LDA version on this node
2023-08-07 22:11:34,365 : INFO : running online (multi-pass) LDA training, 50 topics, 30 passes over the supplied corpus of 10681 documents, updating model once every 2000 documents, evaluating perplexity every 10681 documents, iterating 50x with a convergence threshold of 0.001000
2023-08-07 22:11:34,365 : INFO : PROGRESS: pass 0, at document #2000/10681
2023-08-07 22:11:34,575 : INFO : merging changes from 2000 documents into a model of 10681 documents
2023-08-07 22:11:34,588 : INFO : topic #49 (0.020): 0.036*"pixar" + 0.027*"animation" + 0.020*"disney" + 0.013*"Drama" + 0.010*"classic" + 0.009*"comedy" + 0.008*"Romance" + 0.008*"robin williams" + 0.008*"tom hanks" + 0.008*"Comedy"
2023-08-07 22:11:34,589 : INFO : topic #26 (0.020): 0.024*"james bond" + 0.022*"Comedy" + 0.017*"Drama" + 0.013*

In [7]:
# 토픽의 단어 분포
lda_model.print_topics()

2023-08-07 22:11:54,536 : INFO : topic #26 (0.020): 0.098*"james bond" + 0.064*"vampire" + 0.060*"007" + 0.059*"bond" + 0.056*"assassin" + 0.049*"vampires" + 0.038*"murder" + 0.034*"gothic" + 0.030*"Action" + 0.021*"Thriller"
2023-08-07 22:11:54,537 : INFO : topic #5 (0.020): 0.060*"tim burton" + 0.060*"prison" + 0.050*"friendship" + 0.045*"stephen king" + 0.037*"ummarti2006" + 0.031*"suicide" + 0.026*"nicole kidman" + 0.020*"australian" + 0.020*"beautiful" + 0.018*"dani2006"
2023-08-07 22:11:54,538 : INFO : topic #6 (0.020): 0.139*"Children" + 0.121*"Comedy" + 0.118*"Fantasy" + 0.073*"Adventure" + 0.071*"Musical" + 0.068*"Animation" + 0.034*"christmas" + 0.034*"movie to see" + 0.023*"jim carrey" + 0.020*"fairy tale"
2023-08-07 22:11:54,539 : INFO : topic #33 (0.020): 0.095*"oppl" + 0.078*"revenge" + 0.070*"morgan freeman" + 0.065*"pirates" + 0.048*"hayao miyazaki" + 0.035*"owen wilson" + 0.028*"secret service" + 0.024*"president" + 0.020*"golden raspberry (worst actress)" + 0.018*"gre

[(26,
  '0.098*"james bond" + 0.064*"vampire" + 0.060*"007" + 0.059*"bond" + 0.056*"assassin" + 0.049*"vampires" + 0.038*"murder" + 0.034*"gothic" + 0.030*"Action" + 0.021*"Thriller"'),
 (5,
  '0.060*"tim burton" + 0.060*"prison" + 0.050*"friendship" + 0.045*"stephen king" + 0.037*"ummarti2006" + 0.031*"suicide" + 0.026*"nicole kidman" + 0.020*"australian" + 0.020*"beautiful" + 0.018*"dani2006"'),
 (6,
  '0.139*"Children" + 0.121*"Comedy" + 0.118*"Fantasy" + 0.073*"Adventure" + 0.071*"Musical" + 0.068*"Animation" + 0.034*"christmas" + 0.034*"movie to see" + 0.023*"jim carrey" + 0.020*"fairy tale"'),
 (33,
  '0.095*"oppl" + 0.078*"revenge" + 0.070*"morgan freeman" + 0.065*"pirates" + 0.048*"hayao miyazaki" + 0.035*"owen wilson" + 0.028*"secret service" + 0.024*"president" + 0.020*"golden raspberry (worst actress)" + 0.018*"greed"'),
 (46,
  '0.066*"tom hanks" + 0.051*"mafia" + 0.044*"boring" + 0.042*"overrated" + 0.036*"violent" + 0.032*"eric\'s dvds" + 0.031*"organized crime" + 0.031*"

In [13]:
lda_model.get_topic_terms(0, topn=5)

[(23, 0.12370044),
 (4, 0.08809286),
 (35, 0.062974714),
 (1225, 0.05294164),
 (572, 0.03487635)]

In [8]:
# 문서별 토픽 분포
lda_topics = lda_model[common_corpus]

for i, topic_dist in enumerate(lda_topics):
    topic_dist.sort(key=lambda x: x[1], reverse=True)
    print(topic_dist)

[(49, 0.8754214), (0, 0.034523226), (13, 0.027449576), (10, 0.022539403), (41, 0.017906401)]
[(49, 0.22433497), (8, 0.20574868), (16, 0.17186578), (14, 0.07768817), (10, 0.07747299), (6, 0.06577559), (12, 0.052787222), (32, 0.026154019), (1, 0.026139), (33, 0.026079698), (48, 0.025943212)]
[(10, 0.60123986), (40, 0.12776266), (0, 0.07497296), (4, 0.06800759), (38, 0.06799139)]
[(40, 0.7549999)]
[(49, 0.7509662), (7, 0.101018846), (34, 0.051011685), (4, 0.050991498)]
[(46, 0.25001207), (30, 0.22068843), (36, 0.10415507), (1, 0.10270065), (21, 0.071561545), (24, 0.05715442), (43, 0.054615594), (13, 0.051740266), (2, 0.040429015), (29, 0.026426144)]
[(22, 0.28621), (34, 0.25106224), (19, 0.10101303), (0, 0.061848667), (40, 0.055029076), (36, 0.051008437), (17, 0.051004548), (28, 0.051003587), (7, 0.050810076)]
[(6, 0.60406154), (24, 0.20391743)]
[(2, 0.47075507), (8, 0.28923994)]
[(26, 0.756969), (16, 0.07968903), (18, 0.05917244), (31, 0.02368156), (33, 0.023420688), (10, 0.020205773), (

In [9]:
from util.models import Dataset, RecommendResult
from recommend.base import BaseRecommender
from collections import defaultdict, Counter
import numpy as np
import gensim
import logging
from gensim.corpora.dictionary import Dictionary

np.random.seed(0)


class LdaContentsBasedRecommender(BaseRecommender):
    def recommend(self, dataset: Dataset, k: int, **kwargs) -> RecommendResult:
        n_topics = kwargs.get('n_topics', 50)
        n_epochs = kwargs.get('n_epochs', 30)

        movie_content = dataset.item_content
        movie_content['tag_genre'] = movie_content['tag'].fillna('').apply(list) + movie_content['genre'].fillna('').apply(list)
        tag_genre_data = movie_content.tag_genre.tolist()

        logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

        common_dictionary = Dictionary(tag_genre_data)
        common_corpus = [common_dictionary.doc2bow(doc) for doc in tag_genre_data]

        lda_model = gensim.models.LdaModel(
            corpus=common_corpus,
            num_topics=n_topics,
            id2word=common_dictionary,
            passes=n_epochs
        )

        lda_topics = lda_model[common_corpus]

        movie_topics = []
        movie_topic_scores = []
        for topic_dist in lda_topics:
            sorted_topic_dist = list(sorted(topic_dist, key=lambda x: x[1], reverse=True))
            topic, score = sorted_topic_dist[0]
            movie_topics.append(topic)
            movie_topic_scores.append(score)

        movie_content['topic'] = movie_topics
        movie_content['topic_score'] = movie_topic_scores

        high_rating = dataset.train[dataset.train.rating >= 4]
        user_evalueated_movies = dataset.train.groupby('user_id').agg({'movie_id': list})['movie_id'].to_dict()
        movie_id2index = {
            movie_id: idx for idx, movie_id in enumerate(movie_content.movie_id.tolist())
        }

        pred_user2items = defaultdict(list)

        for user_id, data in high_rating.groupby('user_id'):
            seed_movie_ids = data.sort_values('timestamp')['movie_id'].to_list()[-10:]
            seed_movie_indices = [movie_id2index[movie_id] for movie_id in seed_movie_ids]
            topic_counter = Counter(movie_topics[i] for i in seed_movie_indices)
            freq_topic = topic_counter.most_common(1)[0][0]
            topic_movies = (
                movie_content[movie_content.topic == freq_topic]
                .sort_values('topic_score', ascending=False)
                .movie_id.tolist()
            )

            for movie_id in topic_movies:
                if movie_id not in user_evalueated_movies[user_id]:
                    pred_user2items[user_id].append(movie_id)

                if len(pred_user2items[user_id]) >= k:
                    break

        return RecommendResult(dataset.test.rating, pred_user2items)


In [11]:
recommender = LdaContentsBasedRecommender()
metrics = recommender.run_sample()
print(metrics)

2023-08-07 22:14:39,789 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2023-08-07 22:14:39,855 : INFO : adding document #10000 to Dictionary<14749 unique tokens: ['3d', 'Adventure', 'Animation', 'Children', 'Comedy']...>
2023-08-07 22:14:39,859 : INFO : built Dictionary<15261 unique tokens: ['3d', 'Adventure', 'Animation', 'Children', 'Comedy']...> from 10681 documents (total 117144 corpus positions)
2023-08-07 22:14:39,860 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<15261 unique tokens: ['3d', 'Adventure', 'Animation', 'Children', 'Comedy']...> from 10681 documents (total 117144 corpus positions)", 'datetime': '2023-08-07T22:14:39.860005', 'gensim': '4.3.1', 'python': '3.9.9 (main, May 31 2023, 19:14:41) \n[Clang 14.0.3 (clang-1403.0.22.14.1)]', 'platform': 'macOS-13.2.1-arm64-arm-64bit', 'event': 'created'}
2023-08-07 22:14:39,897 : INFO : using symmetric alpha at 0.02
2023-08-07 22:14:39,898 : INFO : using symmetric eta at 0.02
2023-08-07 22:14:39,8

rmse: 0.000, precision@K: 0.004, recall@K: 0.011
